In [5]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['posts']
posts_collection = db['posts']
posts_collection.drop()




In [71]:
from itertools import islice
import json
from datetime import datetime

with open ('data.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [ ]:
#1.1
import json
import random

from faker import Faker

# Create an instance of the Faker class
fake = Faker()

# Define the fields for posts, comments, and user information
post_fields = ["title", "content"]
comment_fields = ["text", "timestamp"]
user_fields = ["name", "email", "age"]

# Generate a list of 5 users with random information
users = []
for i in range(5):
    user = {}
    user["name"] = fake.name()
    user["email"] = fake.email()
    user["age"] = random.randint(18, 60)
    users.append(user)

# Generate random posts and comments for each user
data = []
for user in users:
    num_posts = random.randint(1, 100)
    for _ in range(num_posts):
        post = {}
        post["user"] = user["name"]
        post["user_email"] = user["email"]
        post["user_age"] = user["age"]
        post["post"] = {}
        post["post"]["title"] = f'Random title for {user["name"]}'
        post["post"]["content"] = f'Random content for {user["name"]} {fake.text()}'
        post["post"]["timestamp"] = fake.date_time().isoformat()
        num_comments = random.randint(0, 5)
        post["comments"] = []
        for _ in range(num_comments):
            comment = {}
            comment["user"] = random.choice(users)["name"]
            comment["user_email"] = random.choice(users)["email"]
            comment["user_age"] = random.choice(users)["age"]
            comment["comment"] = {}
            comment["comment"]["text"] = f"Random text {fake.text()}"
            comment["comment"]["timestamp"] = fake.date_time().isoformat()
            post["comments"].append(comment)
        data.append(post)

for item in data:
    posts_collection.insert_one(item)
    print(item)
    print('\n)')

    

In [ ]:
#1.2
#import pandas as pd
# Buscar todos los usuarios mayores a 25

users = posts_collection.aggregate([
    {"$match": {"user_age": {"$gt": 25}}},
    {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])


# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df


In [ ]:
#1.3
# Buscar un usuario por direccion de correo

user = posts_collection.find_one({"user_email": "tmartin@example.org"})
# Imprimir el resultado
print(user)


In [ ]:
#1.4

In [ ]:
#1.5

In [ ]:
#2.1
# Buscar todos los usuarios cuyo nombre de usuario contenga la palabra S̈mith.̈ (Expresiones regulares)
users = posts_collection.aggregate([
    {"$match": {"user": {"$regex": "Smith", "$options": "i"}}},
    {"$group": {"_id": "$user"}},
    {"$project": {"_id": 0, "user": "$_id"}}
])


# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df


In [ ]:
#2.2

In [ ]:
#2.3

In [ ]:
#2.4

In [ ]:
#2.5

In [ ]:
#3.1
#Buscar usuarios que tengan más de 20 años pero menos de 30. (Combinar operadores)

users = posts_collection.aggregate([
{"$match": {"user_age": {"$gt": 25, "$lt": 30}}},
{"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
{"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

In [ ]:
#3.2

In [ ]:
#3.3

In [ ]:
#3.4

In [ ]:
#3.5

In [ ]:
#4.1
# Ordenar todos los usuarios por edad en orden descendente

users = posts_collection.aggregate([
{"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
{"$sort": {"user_age": -1}},
{"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

In [ ]:
#4.2

In [ ]:
#4.3

In [ ]:
#5.1
#1. Utilizar el pipeline de agregación para calcular el número total de usuarios.

users = posts_collection.aggregate([
{"$group": {"_id": "$user"}},
{"$group": {"_id": None, "Cantidad": {"$sum": 1}}},
{"$project": {"_id": 0, "Cantidad": 1}}
])

df = pd.DataFrame(list(users))
df


In [ ]:
#5.2

In [ ]:
#5.3

In [ ]:
#5.4

In [ ]:
#5.5

In [ ]:
#6.1

# Incrementar la edad de todos los usuarios en 1 con operadores

posts_collection.update_many({}, {"$inc": {"user_age": 1}})

users = posts_collection.aggregate([
    {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])


df = pd.DataFrame(list(users))
df

In [ ]:
#6.2

In [ ]:
#6.3

In [ ]:
#6.4

In [ ]:
#7.1

# Crear un índice en el campo ”user_email” para búsquedas de correo electrónico más rápidas.
posts_collection.create_index([("user_email", 1)])

df = pd.DataFrame(list(posts_collection.index_information()))
df


In [ ]:
#7.2

In [ ]:
#7.3